In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import datetime
import sys
from pathlib import Path

PROJECT_ROOT = Path("/content/forecasting")
sys.path.insert(0, str(PROJECT_ROOT))

import src.features.datawindow as dw
import src.features.model_configuration as mc

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Lambda, Reshape, RNN, LSTMCell


ModuleNotFoundError: No module named 'src'

In [ ]:
url_train = 'https://raw.githubusercontent.com/xbadiam/Forecasting/refs/heads/main/data/output/train.csv'
train_df = pd.read_csv(url_train, index_col=0)
train_df.head(5)

,clouds_all,traffic_volume,day_sin,day_cos
temp,,,,
0.760019,0.0,0.758755,0.146447,0.146447
0.738174,0.0,0.560765,0.066987,0.250000
0.698884,0.0,0.463513,0.017037,0.370590
0.668238,0.0,0.370029,0.000000,0.500000
0.643093,0.0,0.325101,0.017037,0.629410


In [ ]:
url_val = 'https://raw.githubusercontent.com/xbadiam/Forecasting/refs/heads/main/data/output/val.csv'
val_df = pd.read_csv(url_val, index_col=0)
val_df.head(5)

,clouds_all,traffic_volume,day_sin,day_cos
temp,,,,
0.457646,0.01,0.700153,0.370590,0.017037
0.460946,0.20,0.769918,0.250000,0.066987
0.459375,0.05,0.872750,0.146447,0.146447
0.456388,0.20,0.776894,0.066987,0.250000
0.440830,0.05,0.664574,0.017037,0.370590


In [ ]:
url_test = 'https://raw.githubusercontent.com/xbadiam/Forecasting/refs/heads/main/data/output/test.csv'
test_df = pd.read_csv(url_test, index_col=0)
test_df.head(5)

,clouds_all,traffic_volume,day_sin,day_cos
temp,,,,
0.767405,0.9,0.401563,0.000000,0.500000
0.756876,0.9,0.348821,0.017037,0.629410
0.754833,0.9,0.302637,0.066987,0.750000
0.754833,0.9,0.187108,0.146447,0.853553
0.756247,0.9,0.080926,0.250000,0.933013


In [ ]:
train_df = train_df.reset_index()
val_df   = val_df.reset_index()
test_df  = test_df.reset_index()

## Implementing a single-step linear model

We simply take all the inputs, assign a weight to each, take the sum, and generate a prediction. Remember that we are using the traffic volume as a target.

Unlike a baseline model, a linear model actually requires training. Thus, we'll define a compile_and_fit function that configures the model for training and then fits the model on the data, as shown in the following listing.

We'll use the Sequential model from Keras, as it allows us to stack different layers. Since we are building a linear model here, we only have one layer—a Dense layer, which is the most basic layer in deep learning. We’ll specify the number of units as 1, since the model must output only one value: the prediction for traffic volume at the next timestep.

In [ ]:
linear = Sequential([
    Dense(units=1)
])

Clearly, Keras makes it very easy to build models. With this step complete, we can then train the model using compile_and_fit and store the performance to later compare it to the baseline.

In [ ]:
single_step_window = dw.DataWindow(input_width=1, 
                                   label_width=1, 
                                   shift=1, 
                                   label_columns=['traffic_volume'], 
                                   train_df=train_df, val_df=val_df, test_df=test_df)

In [ ]:
val_performance = {}
performance = {}


history = mc.compile_and_fit(linear, single_step_window)

val_performance['Linear'] = linear.evaluate(single_step_window.val)
performance['Linear'] = linear.evaluate(single_step_window.test, verbose=0)